In [8]:
import requests
from lxml import html
import re
import json

In [67]:
re.match("(.*\s)?j((\s.*)?)$"," jd")

<re.Match object; span=(0, 4), match=' j d'>

In [4]:
def calcularPrecio(divs):
    if(len(divs)==0):
        return [-1]
    precios=[]
    for div in divs:
        precio=div.text.strip() if div.text else ""
        precio=precio.replace("-", '0')
        precios.append(precio)
    return precios
def procesarProductos(apartados,base):
    articulosApartados=[]
    for idApartado,apartado in enumerate(apartados):
        if idApartado<1:
            for iCategoria,categoria in enumerate(apartado["categorias"]):
                if(iCategoria<1):
                    for subcategoria in categoria["subcategorias"]:
                        url=subcategoria["url"]
                        if(url[0]=="/"):
                            url=base+url
                        print(url)
                        productos=html.fromstring(requests.get(url).content).xpath("//ul[@class='products-list']/li[@class!='with-contenspot' or not(@class)]")
                        articulos=[]
                        for producto in productos:
                            #Comprobamos si el artículo está agotado
                            agotado=len(producto.xpath("descendant::span[contains(@class,'offline')]"))>0
                            identificacionProducto=producto.xpath("descendant::h2/a")
                            
                            precio=calcularPrecio(producto.xpath("descendant::div[contains(@class,'price-box')]/div"))
                            articulos.append({"nombre":identificacionProducto[0].text.strip(), "url":identificacionProducto[0].attrib.get("href"),"agotado":agotado, "precio":precio})
                            #print(identificacionProducto[0].text.strip() if identificacionProducto and len(identificacionProducto)>0 else "Sin nombre")
                        subcategoria["articulos"]=articulos
    return apartados
#procesarProductos(menus,"https://www.mediamarkt.es")
    
    

In [6]:

def procesarCategorias(urls,base):
    menus=[]
    #print(url)
    for url in urls:
        #url=urls[0]
        if(url[0]=="/"):
            url=base+url
        categoria=html.fromstring(requests.get(url).content)
        menu=categoria.xpath("//div[contains(@class,'categories-flat')]")
        categorias=[]
        nombreCategoria="Desconocido"
        for opcionMenu in menu:
            if re.match("(.*\s)?categories-flat(\s.*)?$",opcionMenu.attrib.get("class"))!=None:
                subcategorias=opcionMenu.xpath("ul[contains(@class,'categories-flat-descendants')]/li")
                for subcategoria in subcategorias:
                    subsubcategorias=subcategoria.xpath("ul/li/a")
                    listaSubcategorias=[]
                    for subsubcategoria in subsubcategorias:
                        listaSubcategorias.append({"nombre":subsubcategoria.text, "url":subsubcategoria.attrib.get("href")})    
                    aSubcategoria=subcategoria.xpath("a")
                    categorias.append({"nombre":aSubcategoria[0].text, "url":aSubcategoria[0].attrib.get("href"), "subcategorias":listaSubcategorias})
            elif re.match("^(.*\s)?categories-flat-current(\s.*)?$",opcionMenu.attrib.get("class"))!=None:
                nombreCategoria=opcionMenu.text
        menus.append({"nombre":nombreCategoria,"categorias":categorias})
        
    return procesarProductos(menus,base)

def preciosMediaMark():
    urlBase="https://www.mediamarkt.es"
    portada=html.fromstring(requests.get(urlBase).content)
    categorias=portada.xpath("//div[contains(@class,'section_grid_services')]//a")
    enlacesCategorias=[]
    for categoria in categorias:
        enlacesCategorias.append(categoria.attrib.get("href"))
    
    return procesarCategorias(enlacesCategorias,urlBase)

articulos=preciosMediaMark()


https://www.mediamarkt.es/es/category/_televisores-701210.html
https://www.mediamarkt.es/es/category/_accesorios-tv-724512.html
https://www.mediamarkt.es/es/category/_soportes-tv-701211.html
https://www.mediamarkt.es/es/category/_internet-tv-tdt-sat%C3%A9lite-701212.html
https://www.mediamarkt.es/es/category/_proyectores-701213.html
https://www.mediamarkt.es/es/category/_reproductores-dvd-blu-ray-701214.html


In [9]:
#articulos[0]
with open('datos/articlos.json', 'w') as outfile:
    json.dump(articulos, outfile)

In [18]:
with open('datos/articulos.csv', 'a',encoding="utf8") as fichero:
    fichero.write("Apartado,categoria,subcategoria,articulo,url,precios\n")
    for apartado in articulos:
        for categoria in apartado["categorias"]:
            for subcategoria in categoria["subcategorias"]:
                if("articulos" in subcategoria):
                    for articulo in subcategoria["articulos"]:
                        fichero.write("\""+apartado["nombre"]+"\",")
                        fichero.write("\""+categoria["nombre"]+"\",")
                        fichero.write("\""+subcategoria["nombre"]+"\",")
                        fichero.write("\""+articulo["nombre"].replace("\"","\"\"")+"\",")
                        fichero.write("\""+articulo["url"]+"\",")
                        precios=""
                        for precio in articulo["precio"]:
                            precio=precio.strip()
                            if(precio!=""):
                                precios+=(", " if len(precios)>0 else "")+precio
                        fichero.write("\""+precios+"\"\n")

                    
                    



In [22]:
preciosMediaMark()

'https://mediamarkt.es/es/category/_tv-y-audio-701116.html'

In [2]:
rss="https://feeds.elpais.com/mrss-s/pages/ep/site/elpais.com/portada"
urls=["https://elpais.com/","https://elpais.com/tecnologia/","https://elpais.com/ciencia/","https://elpais.com/internacional/"]

In [3]:
def scrapArticuloPais(idx, noticias):
    print(idx)
    for i,urlNoticia in enumerate(noticias):
        #print(items[i].text)
        noticia=html.fromstring(requests.get(urlNoticia).content)
        
        tituloNoticia=noticia.xpath("//h1[@id='articulo-titulo']")
        if(len(tituloNoticia)==0):
            tituloNoticia=noticia.xpath("//article//h1")
        #print(tituloNoticia)
        fichero="texto "+str(idx)+"-"+str(i)+".txt"
        titulo=""
        f = open("textos\\"+fichero, "w", encoding="utf8")
        if(len(tituloNoticia)>0):
            #print(tituloNoticia[0].text)
            titulo=tituloNoticia[0].text
            #fichero=titulo+".txt"
        f.write(titulo+"\n")
        entradillaNoticia=noticia.xpath("//h2[@class='articulo-subtitulo']")
        if(len(entradillaNoticia)==0):
            entradillaNoticia=noticia.xpath("//article//h2")
        if(len(entradillaNoticia)>0):
            #print(entradillaNoticia[0].text)
            f.write(entradillaNoticia[0].text+"\n")
        cuerposNoticia=noticia.xpath("//div[@id='cuerpo_noticia']/p")
        if(len(cuerposNoticia)==0):
            cuerposNoticia=noticia.xpath("//article//section[contains(@class,'article_body')]")
        print(cuerposNoticia)
        for i,cuerpo in enumerate(cuerposNoticia):
            if i>0:
                f.write("\n")
            for contenido in cuerpo.itertext():
                #print(cuerpo)
                f.write(contenido)
            
        #print("--------------------")
        f.close()
    print("Terminado",idx)
    

In [4]:
#noticias=html.fromstring(requests.get(rss).content)
#items=noticias.xpath("//item/guid")
idx=0
for url in urls:
    pagina=html.fromstring(requests.get(url).content)
    articulos=pagina.xpath("//article")
    noticias=[]
    for articulo in articulos:
        enlace=articulo.findall("h2/a")
        if(len(enlace)>0):
            href=enlace[0].attrib.get("href")
            if(href[0]=="/"):
                #print(href)
                noticias.append(url+href)
    scrapArticuloPais(idx, noticias)
    idx=idx+1
##print(noticias)

0
[<Element section at 0x2f6b8fba7c8>]
[<Element section at 0x2f6ba1e9e08>]
[<Element section at 0x2f6ba2229a8>]
[<Element section at 0x2f6b8adf2c8>]
[<Element section at 0x2f6b8fba778>]
[<Element p at 0x2f6ba1c68b8>, <Element p at 0x2f6ba1d6368>, <Element p at 0x2f6ba1d84f8>, <Element p at 0x2f6ba1d7b88>, <Element p at 0x2f6ba1d7d18>, <Element p at 0x2f6ba1d7e08>, <Element p at 0x2f6ba1f3098>, <Element p at 0x2f6ba1f3228>, <Element p at 0x2f6ba1f3048>, <Element p at 0x2f6ba1b3368>, <Element p at 0x2f6ba1b31d8>, <Element p at 0x2f6ba1b39a8>, <Element p at 0x2f6ba2285e8>, <Element p at 0x2f6ba2281d8>]
[<Element section at 0x2f6ba1c68b8>]
[<Element section at 0x2f6ba222a48>]
[<Element section at 0x2f6ba1b3638>]
[<Element section at 0x2f6b8fba778>]
[<Element section at 0x2f6b8fba7c8>]
[<Element section at 0x2f6ba1c68b8>]
[<Element section at 0x2f6ba1d6368>]
[<Element section at 0x2f6ba1f3048>]
[<Element section at 0x2f6b8fba7c8>]
[<Element section at 0x2f6ba1f3098>]
[<Element section at 0

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [4]:
pagina=html.fromstring(requests.get("https://www.youtube.com/watch?v=ZWRxEewXjuo").content)
h2=pagina.xpath("//h2[@id='count']")
print(h2)



[]
